In [ ]:
import os.path
import re
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import yt
import glob
import pandas as pd
import math
import scipy.integrate as integrate
from matplotlib.backends.backend_pdf import PdfPages
mpl.rcParams['figure.dpi'] = 300

In [ ]:
def UExact(x,y,t,u0,v0,alpha,beta,A,nu):
    u = u0 - A * beta * math.cos(alpha * (x - u0 * t)) * math.sin(beta * (y - v0 * t)) * math.exp(-(alpha ** 2 + beta ** 2) * nu * t);
    return u


In [ ]:
def VExact(x,y,t,u0,v0,alpha,beta,A,nu):
    v = v0 + A * alpha * math.sin(alpha * (x - u0 * t)) * math.cos(beta * (y - v0 * t)) * math.exp(-(alpha ** 2 + beta ** 2) * nu * t);
    return v


In [ ]:
def natural_sort(l):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split("([0-9]+)", key)]
    return sorted(l, key=alphanum_key)

In [ ]:
color_list = [
    "#EE2E2F",
    "#008C48",
    "#185AA9",
    "#F47D23",
    "#662C91",
    "#A21D21",
    "#B43894",
    "#010202",
]
linestyle_list = [
    "solid",
    "dashed",
    "dotted",
    "dashdot"
]
linewidth_list = [
    1.5,
    0.5
]

In [ ]:
fields = ["x","y","z","velocityy"]

In [ ]:
def postprocess_dist(case,idx,rootdir,methods,grid_types,grid_sizes,
                ax,line_styles,line_colors,line_widths,plotting,line_configuration):
    
    i_method = -1
    for method in methods:
        i_method += 1
        i_grid_type = -1
        for grid_type in grid_types:
            i_grid_type += 1
            i_grid_size = -1
            for grid_size in grid_sizes:
                i_grid_size += 1

                case_dir = os.path.join(root_dir, method, grid_type, case, f'nx_{grid_size}')
                print(case_dir)

                # Load plt files
                plt_files = natural_sort(glob.glob(os.path.join(case_dir, "plt*")))
                
                ds = yt.load(plt_files[idx])
                ray = ds.r[0:,1,0]
                srt = np.argsort(ray["x"])
                df = pd.DataFrame({f: np.array(ray[f][srt]) for f in ["x","velocityy"]})
                t = ds.current_time.value.flatten()[0]
                df["time"] = t
                df["dx"] = [ray.fwidth[i][0].value.flatten()[0] for i in range(len(df["x"].values))]
                
                # Plot scalar distribution
                if plotting == True:
                    if line_configuration == 1:
                        ax.plot(df["x"].values, df["velocityy"].values, label=f'{method}; nx = {grid_size}; grid type = {grid_type}; t = {t}', color=line_colors[i_grid_size], linewidth=line_widths[i_grid_type], linestyle=line_styles[i_method])
                    else:
                        ax.plot(df["x"].values, df["velocityy"].values, label=f'{method}; nx = {grid_size}; grid type = {grid_type}; t = {t}', color=line_colors[i_method], linewidth=line_widths[i_grid_type], linestyle=line_styles[i_grid_size])
    return t, df["x"].values, df["velocityy"].values, ds
            

In [ ]:
error_lists = [[[] for j in range(len(grid_types))] for i in range(len(methods))]
df_lists = [[[[] for k in range(len(grid_sizes))] for j in range(len(grid_types))] for i in range(len(methods))]

i_method = -1
for method in methods:
    i_method += 1
    i_grid_type = -1
    for grid_type in grid_types:
        i_grid_type += 1
        i_grid_size = -1
        for grid_size in grid_sizes:
            i_grid_size += 1

            case_dir = os.path.join(root_dir, method, grid_type, wave_number,f'nx_{grid_size}')
            
            # Load plt files
            plt_files = natural_sort(glob.glob(os.path.join(case_dir, "plt*")))
            
            for idx in range(len(plt_files)):
            
                ds = yt.load(plt_files[idx])
                ray = ds.r[0:,1,0]
                srt = np.argsort(ray["x"])
                df = pd.DataFrame({f: np.array(ray[f][srt]) for f in fields})
                t = ds.current_time.value.flatten()[0]
                df["time"] = t
                df["dx"] = [ray.fwidth[i][0].value.flatten()[0] for i in range(len(df["x"].values))]

                df_lists[i_method][i_grid_type][i_grid_size].append(df)
                
            # Plot error
            error_file = os.path.join(case_dir,'ctv.log')
            with open(error_file) as f:
                lines = f.readlines()
                time = [float(line.split()[0]) for line in lines[1:]]
                u_error = [float(line.split()[1]) for line in lines[1:]]
                v_error = [float(line.split()[2]) for line in lines[1:]]
            data={'time':time,'u_error':u_error,'v_error':v_error}
            error_lists[i_method][i_grid_type].append(pd.DataFrame(data)) 


# Case CTV

In [ ]:
root_dir = '/Users/dbeckers/ctv'

In [ ]:
t_final = 0.5
Lx = 2.0
u0 = 1.0
v0 = 0.0
alpha = 8*math.pi
beta = 8*math.pi
A = 1/(8*math.pi)
nu = 0.001

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
_,_,_,ds = postprocess_dist('8pi',-1,root_dir,['godunov_weno_z'],['uniform'],[128],
                 _,linestyle_list,color_list,[1.0],False,2)
sliceplot = yt.SlicePlot(ds, "z", ("velocityy"))
sliceplot.set_zlim(("velocityy"), -1, 1)
# sliceplot.set_log(("velocityy"), True, symlog_auto=True)
fig = sliceplot.plots[("velocityy")].figure
ax = sliceplot.plots[("velocityy")].axes

def animate(i):
    _,_,_,ds = postprocess_dist('8pi',i,root_dir,['godunov_weno_z'],['uniform'],[128],
                 ax,linestyle_list,color_list,[1.0],False,2)
    sliceplot._switch_ds(ds)
    return 

# FuncAnimation(fig, animate, frames=len(plt_files), interval=200)
anim = FuncAnimation(fig, animate, frames=9, interval=200)
anim.save('2-ctv_case.gif')

# Fine-to-coarse grid 

### Initial timestep

In [ ]:
lw_exact = 0.5
ls_exact = 'dashed'

In [ ]:
_,_,_,ds = postprocess_dist('8pi',0,root_dir,['godunov_weno_z'],['uniform'],[128],
                 _,linestyle_list,color_list,[1.0],False,2)
sliceplot = yt.SlicePlot(ds, "z", ("velocityy"))
sliceplot.set_zlim(("velocityy"), -1, 1)
# sliceplot.set_log(("velocityy"), True, symlog_auto=True)
fig = sliceplot.plots[("velocityy")].figure
ax = sliceplot.plots[("velocityy")].axes
ax.axhline(0,0,1,color="red",linewidth=3)
# fig.savefig('11a-ctv_uniform_final_godunov_ppm_initial_slice.png')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x_exact = np.linspace(0, 2, num=1000)
v_exact = [VExact(x_i,1.0,0.0,u0,v0,alpha,beta,A,nu) for x_i in x_exact]
ax.plot(x_exact,v_exact,color='black',label='exact solution',linewidth=lw_exact,linestyle=ls_exact)

t,_,_,_ = postprocess_dist('8pi',0,root_dir,['godunov_ppm'],['uniform'],[128],
                 ax,linestyle_list,color_list,[1.0],True,2)

ax.legend(loc='upper left')
ax.set_xlim([0,2])
ax.set_ylim([-1.0,1.3])
ax.set_xlabel('x')
ax.set_ylabel('v')

# fig.savefig('12a-ctv_uniform_final_godunov_ppm_initial.png')

### Animated to show dissipation

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
_,_,_,ds = postprocess_dist('8pi',-1,root_dir,['godunov_ppm'],['uniform'],[128],
                 _,linestyle_list,color_list,[1.0],False,2)
sliceplot = yt.SlicePlot(ds, "z", ("velocityy"))
sliceplot.set_zlim(("velocityy"), -1, 1)
# sliceplot.set_log(("velocityy"), True, symlog_auto=True)
fig = sliceplot.plots[("velocityy")].figure
ax = sliceplot.plots[("velocityy")].axes
ax.axhline(0,0,1,color="red",linewidth=3)

def animate(i):
    _,_,_,ds = postprocess_dist('8pi',i,root_dir,['godunov_ppm'],['uniform'],[128],
                 ax,linestyle_list,color_list,[1.0],False,2)
    sliceplot._switch_ds(ds)
    ax.axhline(0,0,1,color="red",linewidth=3)

    return 

# FuncAnimation(fig, animate, frames=len(plt_files), interval=200)
anim = FuncAnimation(fig, animate, frames=9, interval=200)
anim.save('11b-ctv_uniform_final_godunov_ppm_initial_slice.gif')

In [ ]:
plt.rcParams["animation.html"] = "jshtml"

fig = plt.figure()
ax = plt.axes(xlim=(0, 2), ylim=(-1, 1))
line, = ax.plot([], [], color=color_list[0])

def init():
    line.set_data([], [])
    return line,
def animate(i):
    _,x,y,_ = postprocess_dist('8pi',i,root_dir,['godunov_ppm'],['uniform'],[128],
                 ax,linestyle_list,color_list,[1.0],False,2)
    line.set_data(x, y)
    return line,

FuncAnimation(fig, animate, init_func=init, frames=9, interval=200, blit=True)
anim.save('12b-ctv_uniform_final_godunov_ppm_initial.gif')

### Compare methods at final timestep

In [ ]:
_,_,_,ds = postprocess_dist('8pi',-1,root_dir,['godunov_weno_z'],['uniform'],[128],
                 _,linestyle_list,color_list,[1.0],False,2)
sliceplot = yt.SlicePlot(ds, "z", ("velocityy"))
sliceplot.set_zlim(("velocityy"), -1, 1)
# sliceplot.set_log(("velocityy"), True, symlog_auto=True)
fig = sliceplot.plots[("velocityy")].figure
ax = sliceplot.plots[("velocityy")].axes
ax.axhline(0,0,1,color="red",linewidth=3)
fig.savefig('11c-ctv_uniform_final_godunov_weno_z_final_slice.png')
fig

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x_exact = np.linspace(0, 2, num=1000)
v_exact = [VExact(x_i,1.0,0.5,u0,v0,alpha,beta,A,nu) for x_i in x_exact]
ax.plot(x_exact,v_exact,color='black',label='exact solution',linewidth=lw_exact,linestyle=ls_exact)

t,_,_,_ = postprocess_dist('8pi',-1,root_dir,['godunov_ppm','godunov_weno_z','mol_central'],['uniform'],[128],
                 ax,linestyle_list,color_list,[1.0],True,2)

ax.legend(loc='upper left')
ax.set_xlim([0,2])
ax.set_ylim([-1.0,1.3])
ax.set_xlabel('x')
ax.set_ylabel('v')

fig.savefig('12c-ctv_uniform_final_godunov_ppm_weno_z_mol_central.png')
fig

### Zoom

In [ ]:
_,_,_,ds = postprocess_dist('8pi',-1,root_dir,['godunov_weno_z'],['uniform'],[128],
                 _,linestyle_list,color_list,[1.0],False,2)
sliceplot = yt.SlicePlot(ds, "z", ("velocityy"))
sliceplot.set_zlim(("velocityy"), -1, 1)
# sliceplot.set_log(("velocityy"), True, symlog_auto=True)
fig = sliceplot.plots[("velocityy")].figure
ax = sliceplot.plots[("velocityy")].axes
ax.axhline(0,0.5,0.625,color="red",linewidth=3)
fig.savefig('11d-ctv_uniform_final_godunov_weno_z_final_slice_zoom.png')
fig

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x_exact = np.linspace(0, 2, num=1000)
v_exact = [VExact(x_i,1.0,0.5,u0,v0,alpha,beta,A,nu) for x_i in x_exact]
ax.plot(x_exact,v_exact,color='black',label='exact solution',linewidth=lw_exact,linestyle=ls_exact)

t,_,_,_ = postprocess_dist('8pi',-1,root_dir,['godunov_ppm','godunov_weno_z','mol_central'],['uniform'],[128],
                 ax,linestyle_list,color_list,[1.0],True,2)

ax.legend(loc='upper left')
ax.set_xlim([1,1.25])
ax.set_ylim([-1.0,1.3])
ax.set_xlabel('x')
ax.set_ylabel('v')

fig.savefig('12d-ctv_uniform_final_godunov_ppm_weno_z_mol_central_zoom.png')
fig

### Refined

In [ ]:
_,_,_,ds = postprocess_dist('8pi',-1,root_dir,['godunov_weno_z'],['fine_to_coarse'],[64],
                 _,linestyle_list,color_list,[1.0],False,2)
sliceplot = yt.SlicePlot(ds, "z", ("velocityy"))
sliceplot.set_zlim(("velocityy"), -1, 1)
# sliceplot.set_log(("velocityy"), True, symlog_auto=True)
fig = sliceplot.plots[("velocityy")].figure
ax = sliceplot.plots[("velocityy")].axes
ax.axhline(0,0,1,color="red",linewidth=3)
fig.savefig('13a-ctv_refined_final_godunov_weno_z_slice.png')
fig

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x_exact = np.linspace(0, 2, num=1000)
v_exact = [VExact(x_i,1.0,1.0,u0,v0,alpha,beta,A,nu) for x_i in x_exact]
ax.plot(x_exact,v_exact,color='black',label='exact solution',linewidth=lw_exact,linestyle=ls_exact)

t,_,_,_ = postprocess_dist('8pi',-1,root_dir,['godunov_ppm','godunov_weno_z','mol_central'],['fine_to_coarse'],[64],
                 ax,linestyle_list,color_list,[1.0],True,2)

ax.legend(loc='upper left')
ax.set_xlim([0,2])
ax.set_ylim([-1.0,1.3])
ax.set_xlabel('x')
ax.set_ylabel('v')

fig.savefig('14a-ctv_refined_final_godunov_ppm_weno_z_mol_central.png')
fig

### Specifically looking at mol central

In [ ]:
plt.rcParams["animation.html"] = "jshtml"

fig = plt.figure()
ax = plt.axes(xlim=(0, 2), ylim=(-1, 1))
line, = ax.plot([], [], color=color_list[0])

def init():
    line.set_data([], [])
    return line,
def animate(i):
    print(i)
    _,x,y,_ = postprocess_dist('8pi',i,root_dir,['mol_central'],['fine_to_coarse'],[64],
                 ax,linestyle_list,color_list,[1.0],False,2)
    line.set_data(x, y)
    return line,

anim = FuncAnimation(fig, animate, init_func=init, frames=18, interval=200, blit=True)
anim.save('14b-ctv_refined_final_mol_central.gif')

# Modulate with Gaussian

## Viscosity = 1e-3

In [ ]:
_,_,_,ds = postprocess_dist('8pi',0,root_dir,['mol_central'],['fine_to_coarse_modulated'],[64],
                 _,linestyle_list,color_list,[1.0],False,2)
sliceplot = yt.SlicePlot(ds, "z", ("velocityy"))
sliceplot.set_zlim(("velocityy"), -1, 1)
# sliceplot.set_log(("velocityy"), True, symlog_auto=True)
fig = sliceplot.plots[("velocityy")].figure
ax = sliceplot.plots[("velocityy")].axes
ax.axhline(0,0,1,color="red",linewidth=3)
fig.savefig('15a-ctv_refined_modulated_1e-3_final_mol_central_slice_initial.png')
fig

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x_exact = np.linspace(0, 2, num=1000)
v_exact = [VExact(x_i,1.0,1.0,u0,v0,alpha,beta,A,nu) for x_i in x_exact]
ax.plot(x_exact,v_exact,color='black',label='exact solution',linewidth=lw_exact,linestyle=ls_exact)

t,_,_,_ = postprocess_dist('8pi',0,root_dir,['mol_central'],['fine_to_coarse_modulated'],[64],
                 ax,linestyle_list,color_list,[1.0],True,2)

ax.legend(loc='upper left')
ax.set_xlim([0,2])
ax.set_ylim([-1.0,1.3])
ax.set_xlabel('x')
ax.set_ylabel('v')

fig.savefig('16a-ctv_refined_modulated_1e-3_final_mol_central_initial.png')
fig

#### animated

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
_,_,_,ds = postprocess_dist('8pi',-1,root_dir,['mol_central'],['fine_to_coarse_modulated'],[128],
                 _,linestyle_list,color_list,[1.0],False,2)
sliceplot = yt.SlicePlot(ds, "z", ("velocityy"))
sliceplot.set_zlim(("velocityy"), -1, 1)
# sliceplot.set_log(("velocityy"), True, symlog_auto=True)
fig = sliceplot.plots[("velocityy")].figure
ax = sliceplot.plots[("velocityy")].axes
ax.axhline(0,0,1,color="red",linewidth=3)

def animate(i):
    _,_,_,ds = postprocess_dist('8pi',i,root_dir,['mol_central'],['fine_to_coarse_modulated'],[128],
                 ax,linestyle_list,color_list,[1.0],False,2)
    sliceplot._switch_ds(ds)
    ax.axhline(0,0,1,color="red",linewidth=3)

    return 

# FuncAnimation(fig, animate, frames=len(plt_files), interval=200)
anim = FuncAnimation(fig, animate, frames=18, interval=200)
anim.save('15b-ctv_refined_modulated_1e-3_final_mol_central_slice.gif')

In [ ]:
plt.rcParams["animation.html"] = "jshtml"

fig = plt.figure()
ax = plt.axes(xlim=(0, 2), ylim=(-1, 1))
line, = ax.plot([], [], color=color_list[0])

def init():
    line.set_data([], [])
    return line,
def animate(i):
    _,x,y,_ = postprocess_dist('8pi',i,root_dir,['mol_central'],['fine_to_coarse_modulated'],[64],
                 ax,linestyle_list,color_list,[1.0],False,2)
    line.set_data(x, y)
    return line,

anim = FuncAnimation(fig, animate, init_func=init, frames=18, interval=200, blit=True)
anim.save('16b-ctv_refined_modulated_1e-3_final_mol_central.gif')

## Viscosity = 1e-4

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
_,_,_,ds = postprocess_dist('8pi_1e-4',-1,root_dir,['mol_central'],['fine_to_coarse_modulated'],[128],
                 _,linestyle_list,color_list,[1.0],False,2)
sliceplot = yt.SlicePlot(ds, "z", ("velocityy"))
sliceplot.set_zlim(("velocityy"), -1, 1)
# sliceplot.set_log(("velocityy"), True, symlog_auto=True)
fig = sliceplot.plots[("velocityy")].figure
ax = sliceplot.plots[("velocityy")].axes
ax.axhline(0,0,1,color="red",linewidth=3)

def animate(i):
    _,_,_,ds = postprocess_dist('8pi_1e-4',i,root_dir,['mol_central'],['fine_to_coarse_modulated'],[128],
                 ax,linestyle_list,color_list,[1.0],False,2)
    sliceplot._switch_ds(ds)
    ax.axhline(0,0,1,color="red",linewidth=3)

    return 

# FuncAnimation(fig, animate, frames=len(plt_files), interval=200)
anim = FuncAnimation(fig, animate, frames=18, interval=200)
anim.save('15c-ctv_refined_modulated_1e-4_final_mol_central_slice.gif')

In [ ]:
plt.rcParams["animation.html"] = "jshtml"

fig = plt.figure()
ax = plt.axes(xlim=(0, 2), ylim=(-1, 1))
line, = ax.plot([], [], color=color_list[0])

def init():
    line.set_data([], [])
    return line,
def animate(i):
    print(i)
    _,x,y,_ = postprocess_dist('8pi_1e-4',i,root_dir,['mol_central'],['fine_to_coarse_modulated'],[64],
                 ax,linestyle_list,color_list,[1.0],False,2)
    line.set_data(x, y)
    return line,

anim = FuncAnimation(fig, animate, init_func=init, frames=18, interval=200, blit=True)
anim.save('16c-ctv_refined_modulated_1e-4_final_mol_central.gif')

In [ ]:
anim

In [ ]:
sliceplot = yt.SlicePlot(ds, "z", ("velocityy"))
fig = sliceplot.plots[("velocityy")].figure
ax = sliceplot.plots[("velocityy")].axes
ax.axhline(0,0,1,color="red",linewidth=3)

In [ ]:
fig